# 📘 Projeto RAG com LLM Open-Source (Phi-3)

Este notebook apresenta um **pipeline completo de Retrieval-Augmented Generation (RAG)** utilizando um **LLM open-source (Phi-3 Mini)**.

O objetivo é demonstrar, de forma didática, como combinar **múltiplas fontes de dados**, **busca semântica** e **geração de texto** em um único fluxo.

---
### 🧠 O que você verá neste notebook
- Carregamento e uso de um LLM open-source
- Ingestão de dados locais e da web
- Chunking e embeddings de texto
- Busca semântica com FAISS
- Geração de respostas baseada em contexto (RAG)


## 🤖 Etapa 1 — Carregamento do LLM (Phi-3)

Nesta etapa, carregamos o **modelo de linguagem Phi-3 Mini**, responsável pela geração final das respostas.

**Pontos importantes:**
- O modelo é carregado em **GPU** (`device_map="cuda"`) quando disponível
- Utilizamos `float16` para reduzir uso de memória
- O cache é desativado para evitar incompatibilidades do Phi-3 no Colab


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=torch.float16,
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,
    temperature=0.1,
    do_sample=True,
    return_full_text=False,
    use_cache=False  # Evita problemas de compatibilidade na geração
)


## 📦 Etapa 2 — Imports do LangChain

Aqui importamos as ferramentas do **LangChain** responsáveis por:
- Carregar documentos da web
- Representar documentos de forma estruturada
- Dividir textos grandes em chunks menores


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter


## 📄 Etapa 3 — Fonte de Dados Local (TXT)

Nesta etapa, carregamos um **arquivo de texto local**, simulando documentos internos como:
- Políticas
- Manuais
- FAQs

O texto é convertido em um objeto `Document`, que será usado ao longo do pipeline.

In [ ]:
with open("data/politica_reembolso.txt", "r", encoding="utf-8") as file:
    texto_arquivo = file.read()

doc_txt = Document(
    page_content=texto_arquivo,
    metadata={"source": "politica_reembolso.txt"}
)


## 🌐 Etapa 4 — Fonte de Dados Web

Aqui carregamos conteúdo diretamente da **web**, utilizando uma URL pública.

Isso simula cenários reais onde o sistema precisa responder com base em:
- Notícias
- Blogs
- Documentação online


In [ ]:
loader = WebBaseLoader(
    web_paths=("https://www.bbc.com/portuguese/articles/cd19vexw0y1o",)
)

docs_web = loader.load()


## 🔗 Etapa 5 — Unificação das Fontes

Unificamos todas as fontes de dados em uma única lista de documentos.

A partir deste ponto, o pipeline trata **todas as fontes de forma uniforme**, independentemente da origem.

In [ ]:
docs = [doc_txt] + docs_web


## ✂️ Etapa 6 — Chunking dos Documentos

Documentos grandes são divididos em **chunks menores** para:
- Melhorar a busca semântica
- Evitar estouro de contexto no LLM

Utilizamos sobreposição (`overlap`) para preservar continuidade semântica.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    add_start_index=True
)

splits = text_splitter.split_documents(docs)


## 🧠 Etapa 7 — Embeddings + Vector Store (FAISS)

Nesta etapa:
- Cada chunk é convertido em um **vetor numérico (embedding)**
- Os vetores são armazenados em um **índice FAISS**, permitindo busca semântica eficiente


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(
    documents=splits,
    embedding=hf_embeddings
)

print("Vector store criado com sucesso!")
print(f"Total de vetores indexados: {vectorstore.index.ntotal}")


## 🔎 Etapa 8 — Retriever (Busca Semântica)

O **retriever** é responsável por buscar os chunks mais relevantes dado uma pergunta.

Neste exemplo, usamos `k=1` para retornar apenas o chunk mais relevante, reduzindo ruído e custo computacional.

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

query = "Qual é a política de reembolso?"
docs_relevantes = retriever.invoke(query)


## 🤖 Etapa 9 — Prompt + Geração da Resposta (RAG)

Na etapa final, combinamos:
- 📄 Contexto recuperado
- ❓ Pergunta do usuário
- 🤖 LLM

O modelo gera a resposta **baseando-se explicitamente no contexto fornecido**, caracterizando o padrão RAG.

In [ ]:
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Use o contexto para responder a pergunta.
Se não souber, diga que não sabe.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Contexto:
{contexto}

Pergunta:
{pergunta}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

contexto = docs_relevantes[0].page_content

resposta = llm(
    template.format(pergunta=query, contexto=contexto),
    max_new_tokens=150
)

print(resposta[0]["generated_text"])


---
## ✅ Conclusão

Neste notebook, foi construído um **pipeline RAG completo**, integrando:
- Múltiplas fontes de dados
- Busca semântica
- Geração de respostas condicionadas ao contexto

Este padrão é amplamente utilizado em **assistentes inteligentes**, **chatbots corporativos** e **sistemas de busca avançados**.
